# Running ALS on MovieLens (pySpark)

[ALS](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/ml/recommendation.html#ALS) (Alternating Least Squares) is a well-known collaborative filtering algorithm.

This notebook provides an example of how to utilize and evaluate ALS pySpark ML (DataFrame-based API) implementation, meant for large-scale distributed datasets. We use a smaller dataset in this example to run ALS efficiently on Data Science Virtual Machine.

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import numpy as np
from zipfile import ZipFile
import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType, LongType

from reco_utils.common.notebook_utils import is_jupyter, is_databricks
from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.spark_splitters import spark_random_split
from reco_utils.evaluation.spark_evaluation import SparkRatingEvaluation, SparkRankingEvaluation


print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))


System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Spark version: 2.3.1


Set the default parameters.

In [2]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

### 0. Set up Spark context

The following settings work well for debugging locally on VM - change when running on a cluster. We set up a giant single executor with many threads and specify memory cap. 

In [3]:
# the following settings work well for debugging locally on VM - change when running on a cluster
# set up a giant single executor with many threads and specify memory cap
spark = SparkSession \
    .builder \
    .appName("ALS pySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.cores", "32")\
    .config("spark.executor.memory", "8g")\
    .config("spark.memory.fraction", "0.9")\
    .config("spark.memory.stageFraction", "0.3")\
    .config("spark.executor.instances", 1)\
    .config("spark.executor.heartbeatInterval", "36000s")\
    .config("spark.network.timeout", "10000000s")\
    .config("spark.driver.maxResultSize", "50g")\
    .getOrCreate()


### 1. Download the MovieLens dataset

In [4]:
# MovieLens data have different data-format for each size of dataset
data_header = False
if MOVIELENS_DATA_SIZE == "100k":
    separator = "\t"
    data_path = "ml-100k/u.data"
elif MOVIELENS_DATA_SIZE == "1m":
    separator = "::"
    data_path = "ml-1m/ratings.dat"
elif MOVIELENS_DATA_SIZE == "10m":
    separator = "::"
    data_path = "ml-10M100K/ratings.dat"
elif MOVIELENS_DATA_SIZE == "20m":
    separator = ","
    data_path = "ml-20m/ratings.csv"
    data_header = True
else:
    raise ValueError("Invalid data size. Should be one of {100k, 1m, 10m, or 20m}")

# Download dataset zip file and decompress if haven't done yet
dest_folder = "."
dest_file = data_path

if is_databricks():
    # Handle local file I/O APIs on Databricks
    dest_folder = "/dbfs/tmp"
    dest_file = os.path.join(dest_folder, dest_file)
    data_path = "dbfs:/tmp/" + data_path
    
if not os.path.exists(dest_file):
    filename = "ml-" + MOVIELENS_DATA_SIZE + ".zip"
    filepath = maybe_download(
        "http://files.grouplens.org/datasets/movielens/" + filename, filename
    )

    with ZipFile(filepath, "r") as zf:
        zf.extractall(dest_folder)

    # remove zip file we already used
    os.remove(filepath)

# Force the file to be flushed to persistent storage
if is_databricks():
    # In Databricks, passing python variable to shell command like "!sync {dest_file}" does not work.
    if MOVIELENS_DATA_SIZE == "100k":
        !sync "/dbfs/tmp/ml-100k/u.data"
    elif MOVIELENS_DATA_SIZE == "1m":
        !sync "/dbfs/tmp/ml-1m/ratings.dat"
    elif MOVIELENS_DATA_SIZE == "10m":
        !sync "/dbfs/tmp/ml-10M100K/ratings.dat"
    elif MOVIELENS_DATA_SIZE == "20m":
        !sync "/dbfs/tmp/ml-20m/ratings.csv"
else:
    !sync {dest_file}

In [5]:
# Note: The DataFrame-based API for ALS currently only supports integers for user and item ids.
schema = StructType(
    (
        StructField("UserId", IntegerType()),
        StructField("MovieId", IntegerType()),
        StructField("Rating", FloatType()),
        StructField("Timestamp", LongType()),
    )
)

# pySpark's read csv currently doesn't support multi-character delimiter, thus we manually handle that
if len(separator) > 1:
    raw_data = spark.sparkContext.textFile(data_path)
    # In databricks (or maybe in multi-cluster machines), somehow file 
    raw_data.take(1)
    data_rdd = raw_data.map(lambda l: l.split(separator)) \
        .map(lambda c: [int(c[0]), int(c[1]), float(c[2]), int(c[3])])
    data = spark.createDataFrame(data_rdd, schema)
else:
    data = spark.read.csv(data_path, schema=schema, sep=separator, header=data_header)

data.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|   196|    242|   3.0|881250949|
|   186|    302|   3.0|891717742|
|    22|    377|   1.0|878887116|
|   244|     51|   2.0|880606923|
|   166|    346|   1.0|886397596|
|   298|    474|   4.0|884182806|
|   115|    265|   2.0|881171488|
|   253|    465|   5.0|891628467|
|   305|    451|   3.0|886324817|
|     6|     86|   3.0|883603013|
|    62|    257|   2.0|879372434|
|   286|   1014|   5.0|879781125|
|   200|    222|   5.0|876042340|
|   210|     40|   3.0|891035994|
|   224|     29|   3.0|888104457|
|   303|    785|   3.0|879485318|
|   122|    387|   5.0|879270459|
|   194|    274|   2.0|879539794|
|   291|   1042|   4.0|874834944|
|   234|   1184|   2.0|892079237|
+------+-------+------+---------+
only showing top 20 rows



### 2. Split the data using the Spark random splitter provided in utilities

In [6]:
train, test = spark_random_split(data, ratio=0.75, seed=123)
print ("N train", train.cache().count())
print ("N test", test.cache().count())

N train 75193
N test 24807


### 3. Train the ALS model on the training data, and get the top-k recommendations for our testing data

To predict movie ratings, we use the rating data in the training set as users' explicit feedbacks.

When our goal is to recommend top k movies a user is likely to watch, on the other hand, we utilize the ratings as implicit feedbacks.

In [7]:
header = {
    "userCol": "UserId",
    "itemCol": "MovieId",
    "ratingCol": "Rating",
}


# implicitPrefs=True for recommendation, False for rating prediction
als = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=True,
    alpha=0.1,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

In [8]:
model = als.fit(train)


In [9]:
recommendations = model.recommendForUserSubset(test, TOP_K)
recommendations.show()
    

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|   471|[[420, 0.64394724...|
|   463|[[13, 1.0953349],...|
|   833|[[234, 1.2661667]...|
|   496|[[173, 0.75235784...|
|   148|[[169, 0.9539694]...|
|   540|[[50, 0.88092697]...|
|   392|[[50, 0.9353202],...|
|   243|[[275, 1.1422337]...|
|   623|[[127, 0.927937],...|
|   737|[[127, 0.49741036...|
|   897|[[174, 0.99030745...|
|   858|[[127, 0.9081067]...|
|    31|[[269, 0.6397016]...|
|   516|[[357, 0.3626451]...|
|   580|[[405, 1.0172005]...|
|   251|[[121, 1.2434335]...|
|   451|[[289, 1.1142702]...|
|    85|[[275, 1.0268326]...|
|   137|[[117, 1.0683532]...|
|   808|[[313, 1.0408531]...|
+------+--------------------+
only showing top 20 rows



In [10]:
# Convert to reco util's ranking evaluator format
top_k = recommendations.select('UserId', F.explode('recommendations').alias('r')) \
    .select('UserId', 'r.*')
top_k.show()

+------+-------+----------+
|UserId|MovieId|    rating|
+------+-------+----------+
|   471|    420|0.64394724|
|   471|    501|   0.57905|
|   471|    625|0.54104215|
|   471|    404|0.53206724|
|   471|    465|0.52648234|
|   471|    946| 0.4953863|
|   471|    140|0.48872247|
|   471|    416| 0.4879257|
|   471|    102| 0.4875346|
|   471|     99|0.47469372|
|   463|     13| 1.0953349|
|   463|    124| 1.0937009|
|   463|    302| 1.0562845|
|   463|    286|0.97413903|
|   463|    285| 0.9637387|
|   463|    242| 0.9326599|
|   463|      1| 0.9270849|
|   463|     19| 0.9269906|
|   463|    303|0.90075296|
|   463|    269|0.89880556|
+------+-------+----------+
only showing top 20 rows



### 4. Evaluate how well ALS performs

In [11]:
test.show()

+------+-------+------+---------+
|UserId|MovieId|Rating|Timestamp|
+------+-------+------+---------+
|     1|      2|   3.0|876893171|
|     1|      3|   4.0|878542960|
|     1|      4|   3.0|876893119|
|     1|      9|   5.0|878543541|
|     1|     11|   2.0|875072262|
|     1|     17|   3.0|875073198|
|     1|     25|   4.0|875071805|
|     1|     28|   4.0|875072173|
|     1|     30|   3.0|878542515|
|     1|     33|   4.0|878542699|
|     1|     43|   4.0|878542869|
|     1|     48|   5.0|875072520|
|     1|     49|   3.0|878542478|
|     1|     52|   4.0|875072205|
|     1|     59|   5.0|876892817|
|     1|     62|   3.0|878542282|
|     1|     65|   4.0|875072125|
|     1|     66|   4.0|878543030|
|     1|     71|   3.0|876892425|
|     1|     78|   1.0|878543176|
+------+-------+------+---------+
only showing top 20 rows



In [12]:
rank_eval = SparkRankingEvaluation(test, top_k, k = TOP_K, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="rating", 
                                    relevancy_method="top_k")

In [13]:
print("Model:\tALS",
      "Top K:\t%d" % rank_eval.k,
      "MAP:\t%f" % rank_eval.map_at_k(),
      "NDCG:\t%f" % rank_eval.ndcg_at_k(),
      "Precision@K:\t%f" % rank_eval.precision_at_k(),
      "Recall@K:\t%f" % rank_eval.recall_at_k(), sep='\n')

Model:	ALS
Top K:	10
MAP:	0.025732
NDCG:	0.099981
Precision@K:	0.094268
Recall@K:	0.075972


### 5. Evaluate rating prediction

In [14]:
als_prediction = ALS(
    rank=40,
    maxIter=15,
    implicitPrefs=False,
    regParam=0.01,
    coldStartStrategy='drop',
    nonnegative=True,
    **header
)

model_prediction = als_prediction.fit(train)

prediction = model_prediction.transform(test)
prediction.show()


+------+-------+------+---------+----------+
|UserId|MovieId|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   406|    148|   3.0|879540276|  3.149721|
|    27|    148|   3.0|891543129| 3.1062589|
|   606|    148|   3.0|878150506|  4.206346|
|   916|    148|   2.0|880843892| 2.0281613|
|   236|    148|   4.0|890117028| 3.4453466|
|   602|    148|   4.0|888638517| 3.7288864|
|   663|    148|   4.0|889492989| 3.2095706|
|   372|    148|   5.0|876869915| 4.9773664|
|   190|    148|   4.0|891033742|  4.093347|
|     1|    148|   2.0|875240799| 1.5550745|
|   297|    148|   3.0|875239619|  2.721611|
|   178|    148|   4.0|882824325|  3.561745|
|   308|    148|   3.0|887740788| 3.5785131|
|   923|    148|   4.0|880387474|  4.117514|
|    54|    148|   3.0|880937490|  3.801107|
|   430|    148|   2.0|877226047| 4.4093704|
|    92|    148|   2.0|877383934| 2.7917807|
|   447|    148|   4.0|878854729| 3.7435863|
|   374|    148|   4.0|880392992| 3.5979629|
|   891|  

In [15]:
rating_eval = SparkRatingEvaluation(test, prediction, col_user="UserId", col_item="MovieId", 
                                    col_rating="Rating", col_prediction="prediction")

print("Model:\tALS rating prediction",
      "RMSE:\t%.2f" % rating_eval.rmse(),
      "MAE:\t%f" % rating_eval.mae(),
      "Explained variance:\t%f" % rating_eval.exp_var(),
      "R squared:\t%f" % rating_eval.rsquared(), sep='\n')

Model:	ALS rating prediction
RMSE:	1.11
MAE:	0.869224
Explained variance:	0.020389
R squared:	0.016262


In [16]:
if is_jupyter():
    # Record results with papermill for tests
    import papermill as pm
    pm.record("map", rank_eval.map_at_k())
    pm.record("ndcg", rank_eval.ndcg_at_k())
    pm.record("precision", rank_eval.precision_at_k())
    pm.record("recall", rank_eval.recall_at_k())
    pm.record("rmse", rating_eval.rmse())
    pm.record("mae", rating_eval.mae())
    pm.record("exp_var", rating_eval.exp_var())
    pm.record("rsquared", rating_eval.rsquared())
